# Orthogonality, Gram-Schmidt algorithm and QR decomposition
<hr>
This notebook introduces the concept of orthogonality matrices. We will show how to create and how to check whether a matrix is orthogonal or not. Then we implement the QR decomposition to build an orthonormal basis, starting from a generic basis. For the QR decomposition we implemented both the Standard and the Modified Gram-Schimidt procedure, which is more stable than the basic algorithm. Then, we will compare the QR decomposition given by the implemented procedure with the procedure in the package, which use the Householder transformation. Lastly, we will also show the QR decompostion with pivoting.

## Import libraries

In [1]:
import scipy as sp
import numpy as np
import scipy.stats as sst
import scipy.linalg as spl 

## Orthogonal matrices: General check

Create an orthogonal random matrix using the statistical package of scipy. 
<b>Orthogonal</b> matrices are <b>square matrices</b>, so we can select only one dimension.

In [2]:
M = sst.ortho_group.rvs(dim=3) 
print('Orthogonal Random matrix of size 3x3: \n', M)

Orthogonal Random matrix of size 3x3: 
 [[-0.48344283  0.78704386 -0.38320359]
 [-0.43502316  0.16386988  0.88538213]
 [ 0.75963009  0.59473408  0.26316059]]


We can check if a matrix is an orthogonal matrix by checking the main property of orthogonal matrices: 
* the product between an orthogonal matrix and its transpose leads to the identity matrix ( $M M^T = M^TM = I$ )
* the inverse of an orthogonal matrix is equal to the transpose of the matrix itself ( $M^T = M^{-1}$ )
* the determinant of an orthogonal matrix is always equal to 1 or -1

In [3]:
print('dot product M*M^T = I: \n', np.dot(M,M.T))
print('\ndot product M^T*M = I: \n', np.dot(M.T,M))
print('\nWhat we obtain is the identity matrix. On the main diagonal we obtain 1 in both matrices.\n'
      'In all the other position we have elements related to the machine precision. So, even if theoretically\n'
      'we don\'t obtain the identity matrix, numerically these elements are related to machine precision and so\n'
      'we can say that we obtain the identity matrix.')

dot product M*M^T = I: 
 [[ 1.00000000e+00  2.39822332e-16 -7.24669874e-17]
 [ 2.39822332e-16  1.00000000e+00  8.79790241e-17]
 [-7.24669874e-17  8.79790241e-17  1.00000000e+00]]

dot product M^T*M = I: 
 [[ 1.00000000e+00  5.57173139e-17 -7.13835139e-17]
 [ 5.57173139e-17  1.00000000e+00  4.02157739e-16]
 [-7.13835139e-17  4.02157739e-16  1.00000000e+00]]

What we obtain is the identity matrix. On the main diagonal we obtain 1 in both matrices.
In all the other position we have elements related to the machine precision. So, even if theoretically
we don't obtain the identity matrix, numerically these elements are related to machine precision and so
we can say that we obtain the identity matrix.


In [4]:
print('Determinant of the matrix M: ', sp.linalg.det(M)) # det = -1 or det = 1

Determinant of the matrix M:  0.9999999999999998


#### Generic random matrix
Create a generic random matrix with uniform distribution and check that the given matrix is not orthogonal.

In [5]:
A= np.random.rand(5,3)
print('Generic Random matrix: \n',A)

Generic Random matrix: 
 [[0.41666198 0.89583484 0.28698528]
 [0.01193928 0.27755145 0.60142322]
 [0.30357882 0.72338563 0.04746971]
 [0.82879544 0.03098105 0.61571325]
 [0.6623613  0.69922799 0.20292096]]


In [6]:
print('Rank of A:', np.linalg.matrix_rank(A))
print('dot product M*M^T =/= I: \n', np.dot(A,A.T))
print('\ndot product M^T*M =/= I: \n', np.dot(A.T,A))

Rank of A: 3
dot product M*M^T =/= I: 
 [[1.05848781 0.42621452 0.78814691 0.54978209 0.96060889]
 [0.42621452 0.43888725 0.23295064 0.3887983  0.32402124]
 [0.78814691 0.23295064 0.61770025 0.30324372 0.71652294]
 [0.54978209 0.3887983  0.30324372 1.06696452 0.69556597]
 [0.96060889 0.32402124 0.71652294 0.69556597 0.96881919]]

dot product M^T*M =/= I: 
 [[1.39153423 1.08499715 0.78587455]
 [1.08499715 1.89272125 0.61931967]
 [0.78587455 0.61931967 0.86660354]]


### QR decomposition using Standard Gram-Schmidt procedure
The QR decomposition is a factorization of type A = QR, where Q(U) is an orthogonal matrix and R is an upper triangular matrix, if A is square. The QR factorization implemented uses the Standard Gram-Schmidt procedure.
Gram-Schmidt procedure used to orthogonalize a set of <b>linearly independent </b> vectors or columns of a matrix. This procedure gives in output the QR factorization related to the matrix X. <br>
Gram-Schmidt algorithm is defined in the following way:
* Compute the first orthonormal vector $u_1 = \frac{x_1}{\alpha_1}$, where $\alpha_1 = \|x_i\|$
* Compute the k+1 orthonormal vector $u_{k+1} = \frac{z_{k+1}}{\alpha_{k+1}}$, where $\alpha_{k+1} = \|z_{k+1}\|$ and $z_{k+1}= x_{k+1} - \sum_{i=1}^{k} <u_{i} | x_{k+1}>u_{i} $<br>

Input and Output parameters:
* INPUT 
  * <b>X</b>: a matrix for which the columns define a set of linearly independent vectors
* OUTPUT 
  * <b>U</b>: the orthogonal matrix, for which the columns define an orthonormal set of vectors
  * <b>R</b>: the upper triangular matrix which collects the coefficients $\alpha^{(k)}_i$ 

In [7]:
def Gram_Schmidt(X):
    (m,n) = X.shape
    U = np.zeros((m,n))
    R = np.zeros((n,n))
    
    for i in range(n):  # this loop selects the columns of the input matrix, which are the set of linear independent vectors
        U[:,i] = X[:,i] # at each iteration, we take the vector x_k+1 needed to compute z_k+1
        for j in range(i):
            R[j,i] = np.inner(X[:,i], U[:,j]) # compute the inner product <u_i | x_k+1>
            U[:,i] = U[:,i] - R[j,i]*U[:,j]   # compute z_k+1 = x_k+1 - <u_i | x_k+1>*u_i
        R[i,i]= np.linalg.norm(U[:,i],2)    # compute alpha = || z_k+1 ||  
        U[:,i] = U[:,i]/R[i,i] # compute u_i = z_k+1 / alpha
    return U,R

#### Example 1

In [8]:
A = np.array([[1,0,-3,0,2],[0,1,-2,0,1],[0,0,0,1,-1]])
A=A.T
print('Matrix A:\n', A)

Matrix A:
 [[ 1  0  0]
 [ 0  1  0]
 [-3 -2  0]
 [ 0  0  1]
 [ 2  1 -1]]


In [9]:
(U,R) = Gram_Schmidt(A)
print('\nQR factorization of A using standard Gram-Schmidt procedure:')
print('Matrix U =\n',U)
print('\nMatrix R =\n',R)


QR factorization of A using standard Gram-Schmidt procedure:
Matrix U =
 [[ 0.26726124 -0.47809144  0.153393  ]
 [ 0.          0.83666003 -0.0766965 ]
 [-0.80178373 -0.23904572 -0.306786  ]
 [ 0.          0.          0.76696499]
 [ 0.53452248 -0.11952286 -0.53687549]]

Matrix R =
 [[ 3.74165739  2.13808994 -0.53452248]
 [ 0.          1.19522861  0.11952286]
 [ 0.          0.          1.30384048]]


In [10]:
print('dot product U^T*U == I, the columns are an orthonormal set: \n', np.dot(U.T,U))
print('\ndot product U*U^T =/= I, the rows are not an orthonormal set: \n', np.dot(U,U.T))
print('\nSince the matrix is not square, only U^T*U is equal to the identity. '
      'If we compute U*U^T is different from the identity.\nU is NOT an orthogonal matrix. '
      'An easy way to check is computing the norm of the residuals.')

dot product U^T*U == I, the columns are an orthonormal set: 
 [[ 1.00000000e+00 -1.58953834e-16  7.71309326e-17]
 [-1.58953834e-16  1.00000000e+00 -8.02189320e-17]
 [ 7.71309326e-17 -8.02189320e-17  1.00000000e+00]]

dot product U*U^T =/= I, the rows are not an orthonormal set: 
 [[ 0.32352941 -0.41176471 -0.14705882  0.11764706  0.11764706]
 [-0.41176471  0.70588235 -0.17647059 -0.05882353 -0.05882353]
 [-0.14705882 -0.17647059  0.79411765 -0.23529412 -0.23529412]
 [ 0.11764706 -0.05882353 -0.23529412  0.58823529 -0.41176471]
 [ 0.11764706 -0.05882353 -0.23529412 -0.41176471  0.58823529]]

Since the matrix is not square, only U^T*U is equal to the identity. If we compute U*U^T is different from the identity.
U is NOT an orthogonal matrix. An easy way to check is computing the norm of the residuals.


In [11]:
print('||U^T*U - I|| =', np.linalg.norm(np.dot(U.T,U)-np.eye(3)))
print('||U*U^T - I|| =', np.linalg.norm(np.dot(U,U.T)-np.eye(5)))

||U^T*U - I|| = 2.960187663962567e-16
||U*U^T - I|| = 1.4142135623730951


#### Example 2

In [12]:
A = np.array( [[1, 0, 0, 1 ],[0,0,1,-1],[4,0,1,-3]])
A=A.T
print('Matrix A:\n', A)

Matrix A:
 [[ 1  0  4]
 [ 0  0  0]
 [ 0  1  1]
 [ 1 -1 -3]]


In [13]:
(U,R) = Gram_Schmidt(A)
print('\nQR factorization of A using standard Gram-Schmidt procedure:')
print('Matrix U =\n',U)
print('\nMatrix R =\n',R)


QR factorization of A using standard Gram-Schmidt procedure:
Matrix U =
 [[ 0.70710678  0.40824829  0.57735027]
 [ 0.          0.          0.        ]
 [ 0.          0.81649658 -0.57735027]
 [ 0.70710678 -0.40824829 -0.57735027]]

Matrix R =
 [[ 1.41421356 -0.70710678  0.70710678]
 [ 0.          1.22474487  3.67423461]
 [ 0.          0.          3.46410162]]


In [14]:
print('dot product U^T*U == I, the columns are an orthonormal set: \n', np.dot(U.T,U))
print('\ndot product U*U^T =/= I, the rows are not an orthonormal set: \n', np.dot(U,U.T))
print('\nSince the matrix is not square, only U^T*U is equal to the identity. '
      'If we compute U*U^T is different from the identity.\nU is NOT an orthogonal matrix. '
      'An easy way to check is computing the norm of the residuals.')

dot product U^T*U == I, the columns are an orthonormal set: 
 [[ 1.00000000e+00 -1.25760346e-16  1.39213053e-16]
 [-1.25760346e-16  1.00000000e+00 -2.14036242e-16]
 [ 1.39213053e-16 -2.14036242e-16  1.00000000e+00]]

dot product U*U^T =/= I, the rows are not an orthonormal set: 
 [[ 1.00000000e+00  0.00000000e+00 -1.72174929e-16 -9.09009624e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.72174929e-16  0.00000000e+00  1.00000000e+00 -1.22556038e-16]
 [-9.09009624e-17  0.00000000e+00 -1.22556038e-16  1.00000000e+00]]

Since the matrix is not square, only U^T*U is equal to the identity. If we compute U*U^T is different from the identity.
U is NOT an orthogonal matrix. An easy way to check is computing the norm of the residuals.


In [15]:
print('||U^T*U - I|| =', np.linalg.norm(np.dot(U.T,U)-np.eye(3)))
print('||U*U^T - I|| =', np.linalg.norm(np.dot(U,U.T)-np.eye(4)))

||U^T*U - I|| = 5.567102687959989e-16
||U*U^T - I|| = 1.0


#### Example 3

In [16]:
A = np.array( [[1, 0, 0, 1 ],[0,0,1,-1],[4,0,1,3]])
A=A.T
print('Matrix A:\n', A)

Matrix A:
 [[ 1  0  4]
 [ 0  0  0]
 [ 0  1  1]
 [ 1 -1  3]]


In [17]:
(U,R) = Gram_Schmidt(A)
print('\nQR factorization of A using standard Gram-Schmidt procedure:')
print('Matrix U =\n',U)
print('\nMatrix R =\n',R)


QR factorization of A using standard Gram-Schmidt procedure:
Matrix U =
 [[ 0.70710678  0.40824829  0.80949051]
 [ 0.          0.          0.        ]
 [ 0.          0.81649658  0.231283  ]
 [ 0.70710678 -0.40824829  0.53966034]]

Matrix R =
 [[ 1.41421356e+00 -7.07106781e-01  4.94974747e+00]
 [ 0.00000000e+00  1.22474487e+00  1.22474487e+00]
 [ 0.00000000e+00  0.00000000e+00  1.44008380e-15]]


In [18]:
print('In this case we can check that in the matrix R we obtained 10e-15. Why?\n'
      'This happens because Gram-Schmidt works only if the set of vectors is linearly independent.\n'
      'We can check the rank to be sure that the matrix is not full-rank.')
print('Rank of A:', np.linalg.matrix_rank(A))

In this case we can check that in the matrix R we obtained 10e-15. Why?
This happens because Gram-Schmidt works only if the set of vectors is linearly independent.
We can check the rank to be sure that the matrix is not full-rank.
Rank of A: 2


In [19]:
print('dot product U^T*U =/= I, the columns are not an orthonormal set: \n', np.dot(U.T,U))
print('\ndot product U*U^T =/= I, the rows are not an orthonormal set: \n', np.dot(U,U.T))
print('\nThe matrix is not square and the set of vectors are not linearly independent.\nWe are not able to retrieve the identity.\n'
      'An easy way to check is computing the norm of the residuals.')

dot product U^T*U =/= I, the columns are not an orthonormal set: 
 [[ 1.00000000e+00 -1.25760346e-16  9.53993717e-01]
 [-1.25760346e-16  1.00000000e+00  2.98999487e-01]
 [ 9.53993717e-01  2.98999487e-01  1.00000000e+00]]

dot product U*U^T =/= I, the rows are not an orthonormal set: 
 [[ 1.32194156  0.          0.52055473  0.77018326]
 [ 0.          0.          0.          0.        ]
 [ 0.52055473  0.          0.72015849 -0.20851907]
 [ 0.77018326  0.         -0.20851907  0.95789995]]

The matrix is not square and the set of vectors are not linearly independent.
We are not able to retrieve the identity.
An easy way to check is computing the norm of the residuals.


In [20]:
print('||U^T*U - I|| =', np.linalg.norm(np.dot(U.T,U)-np.eye(3)))
print('||U*U^T - I|| =', np.linalg.norm(np.dot(U,U.T)-np.eye(4)))

||U^T*U - I|| = 1.413863292754751
||U*U^T - I|| = 1.731764825430782


#### Example 4

In [21]:
B = np.random.rand(3,3)
print('Matrix B:\n', B)

Matrix B:
 [[0.44624462 0.8878012  0.85143176]
 [0.5441883  0.42121463 0.34588257]
 [0.75972005 0.34181836 0.34844856]]


In [22]:
(UB,RB) = Gram_Schmidt(B)
print('\nQR factorization of B using standard Gram-Schmidt procedure:')
print('Matrix U =\n',UB)
print('\nMatrix R =\n',RB)


QR factorization of B using standard Gram-Schmidt procedure:
Matrix U =
 [[ 0.43090797  0.87564789  0.21808047]
 [ 0.52548549 -0.04702355 -0.84950208]
 [ 0.73360979 -0.48065534  0.48040309]]

Matrix R =
 [[1.03559148 0.85466408 0.80427028]
 [0.         0.59329742 0.56180614]
 [0.         0.         0.05924844]]


In [23]:
print('dot product U^T*U == I: \n', np.dot(UB.T,UB))
print('\ndot product U*U^T == I: \n', np.dot(UB,UB.T))

dot product U^T*U == I: 
 [[1.00000000e+00 3.33224059e-17 6.19683798e-16]
 [3.33224059e-17 1.00000000e+00 1.36581875e-15]
 [6.19683798e-16 1.36581875e-15 1.00000000e+00]]

dot product U*U^T == I: 
 [[ 1.00000000e+00 -1.17053314e-15  8.25450186e-16]
 [-1.17053314e-15  1.00000000e+00  3.04706413e-16]
 [ 8.25450186e-16  3.04706413e-16  1.00000000e+00]]


In [24]:
print('B-np.dot(UB,RB):\n', B-np.dot(UB,RB))

B-np.dot(UB,RB):
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


### QR decomposition using the Householder transformation
We use the implemented method in a lapack library, written in Fortran.


#### Example 1

In [25]:
(Q,R)=np.linalg.qr(B)
print('\nQR factorization of B using Householder transformation:')
print('Matrix Q =\n',Q)
print('\nMatrix R =\n',R)


QR factorization of B using Householder transformation:
Matrix Q =
 [[-0.43090797  0.87564789  0.21808047]
 [-0.52548549 -0.04702355 -0.84950208]
 [-0.73360979 -0.48065534  0.48040309]]

Matrix R =
 [[-1.03559148 -0.85466408 -0.80427028]
 [ 0.          0.59329742  0.56180614]
 [ 0.          0.          0.05924844]]


In [26]:
print('The QR decomposition of the matrix B using the Gram-Schmidt procedure gives the\n'
      'the same values with different sign. So the QR decomposition is not totally unique.\n'
      'Sometimes the sign could be different but the values are the same.\n')
print('Matrix U =\n',UB)
print('\nMatrix R =\n',RB)

The QR decomposition of the matrix B using the Gram-Schmidt procedure gives the
the same values with different sign. So the QR decomposition is not totally unique.
Sometimes the sign could be different but the values are the same.

Matrix U =
 [[ 0.43090797  0.87564789  0.21808047]
 [ 0.52548549 -0.04702355 -0.84950208]
 [ 0.73360979 -0.48065534  0.48040309]]

Matrix R =
 [[1.03559148 0.85466408 0.80427028]
 [0.         0.59329742 0.56180614]
 [0.         0.         0.05924844]]


#### Example 2
In this example, we use again the matrix A, which wasn't full rank to show that the QR decomposition using the Householder transformation works also with matrix that has not full rank.

In [27]:
(Q,R)=np.linalg.qr(A)
print('\nQR factorization (Thin QR) of A using Householder transformation:')
print('Matrix Q =\n',Q)
print('\nMatrix R =\n',R)


QR factorization (Thin QR) of A using Householder transformation:
Matrix Q =
 [[-0.70710678 -0.40824829 -0.09763107]
 [-0.          0.          0.98559856]
 [-0.         -0.81649658  0.09763107]
 [-0.70710678  0.40824829  0.09763107]]

Matrix R =
 [[-1.41421356e+00  7.07106781e-01 -4.94974747e+00]
 [ 0.00000000e+00 -1.22474487e+00 -1.22474487e+00]
 [ 0.00000000e+00  0.00000000e+00  3.14018492e-16]]


In [28]:
print('dot product Q^T*Q == I: \n', np.dot(Q.T,Q))

dot product Q^T*Q == I: 
 [[ 1.00000000e+00 -4.72557231e-17 -2.13246765e-17]
 [-4.72557231e-17  1.00000000e+00 -2.86325893e-16]
 [-2.13246765e-17 -2.86325893e-16  1.00000000e+00]]


### QR decomposition with pivoting

#### Example 1

In [29]:
[Qp,Rp,Pp]=spl.qr(A,pivoting=True)
print('\nQR factorization of A using pivoting:')
print('Matrix Q =\n',Qp)
print('\nMatrix R =\n',Rp)
print('\nPermutation vector P =', Pp)


QR factorization of A using pivoting:
Matrix Q =
 [[-0.78446454 -0.22645541 -0.44323528 -0.36996732]
 [-0.          0.          0.64080219 -0.76770603]
 [-0.19611614 -0.79259392  0.44323528  0.36996732]
 [-0.58834841  0.56613852  0.44323528  0.36996732]]

Matrix R =
 [[-5.09901951e+00  3.92232270e-01 -1.37281295e+00]
 [ 0.00000000e+00 -1.35873244e+00  3.39683110e-01]
 [ 0.00000000e+00  0.00000000e+00 -8.32667268e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

Permutation vector P = [2 1 0]


#### Example 2

In [30]:
AT = np.array([[1, 1e-16,1e-16],[1, 1e-16, 0], [1, 0, 1e-16]])
A = AT.T
print('Matrix A:\n', A)

Matrix A:
 [[1.e+00 1.e+00 1.e+00]
 [1.e-16 1.e-16 0.e+00]
 [1.e-16 0.e+00 1.e-16]]


In [31]:
(U,R) = Gram_Schmidt(A)
print('Matrix Q =\n',Q)
print('\nMatrix R =\n',R)
print('\n||U*R-A|| =', np.linalg.norm(np.dot(U,R)-A,2))
print('||U^T*U-I|| =',np.linalg.norm(np.dot(U.T,U)-np.eye(3),2))
print('||U*U^T-I|| =',np.linalg.norm(np.dot(U,U.T)-np.eye(3),2))

Matrix Q =
 [[-0.70710678 -0.40824829 -0.09763107]
 [-0.          0.          0.98559856]
 [-0.         -0.81649658  0.09763107]
 [-0.70710678  0.40824829  0.09763107]]

Matrix R =
 [[ 1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e-16 -1.00000000e-16]
 [ 0.00000000e+00  0.00000000e+00  1.41421356e-16]]

||U*R-A|| = 3.758512058236175e-33
||U^T*U-I|| = 0.7071067811865477
||U*U^T-I|| = 0.7071067811865476


In [32]:
print('dot product U^T*U =\= I: \n', np.dot(U.T,U))
print('\ndot product U*U^T =\= I: \n', np.dot(U,U.T))

dot product U^T*U =\= I: 
 [[ 1.00000000e+00 -1.00000000e-16 -1.41421356e-16]
 [-1.00000000e-16  1.00000000e+00  7.07106781e-01]
 [-1.41421356e-16  7.07106781e-01  1.00000000e+00]]

dot product U*U^T =\= I: 
 [[1.0e+00 1.0e-16 1.0e-16]
 [1.0e-16 5.0e-01 5.0e-01]
 [1.0e-16 5.0e-01 1.5e+00]]


### QR decomposition using Modified Gram-Schmidt procedure

In [33]:
def Modified_Gram_Schmidt(X):
    (m,n) = X.shape
    U = np.copy(X)
    R = np.zeros((n,n))
   
    R[0,0]= np.linalg.norm(U[:,0],2) 
    U[:,0] = U[:,0]/R[0,0] #  orthonormal vectors

    for i in range(1,n):# loop on the columns, the vectors are linearly independent
        for j in range(i,n):
            R[i-1,j] = np.inner(U[:,j], U[:,i-1])
            U[:,j] = U[:,j] - R[i-1,j]*U[:,i-1]
        R[i,i]= np.linalg.norm(U[:,i],2) 
        U[:,i] = U[:,i]/R[i,i] # orthonormal vectors
       
    return (U,R)

#### Example 1 
We will compute the QR decomposition again on the matrix A, for which the QR decomposition using the Standard Gram-Schmidt doesn't work. Now we want to show that using the Modified Gram-Schmidt everything works well.

In [34]:
(U,R) = Modified_Gram_Schmidt(A)
print('Matrix Q =\n',Q)
print('\nMatrix R =\n',R)
print('\n||U*R-A|| =', np.linalg.norm(np.dot(U,R)-A,2))
print('||U^T*U-I|| =',np.linalg.norm(np.dot(U.T,U)-np.eye(3),2))
print('||U*U^T-I|| =',np.linalg.norm(np.dot(U,U.T)-np.eye(3),2))

Matrix Q =
 [[-0.70710678 -0.40824829 -0.09763107]
 [-0.          0.          0.98559856]
 [-0.         -0.81649658  0.09763107]
 [-0.70710678  0.40824829  0.09763107]]

Matrix R =
 [[1.e+00 1.e+00 1.e+00]
 [0.e+00 1.e-16 0.e+00]
 [0.e+00 0.e+00 1.e-16]]

||U*R-A|| = 0.0
||U^T*U-I|| = 1.4142135623730949e-16
||U*U^T-I|| = 1.4142135623730949e-16


In [35]:
print('dot product U^T*U == I: \n', np.dot(U.T,U))
print('\nA-np.dot(U,R): \n', A-np.dot(U,R))

dot product U^T*U == I: 
 [[ 1.e+00 -1.e-16 -1.e-16]
 [-1.e-16  1.e+00  0.e+00]
 [-1.e-16  0.e+00  1.e+00]]

A-np.dot(U,R): 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


#### Example 2
Now we take a random matrix, and we compare the QR decomposition obtained using both Standard and Modified Gram-Schmidt. Since the matrix is random and the columns are linearly independent, then both methods work well.

In [36]:
A=np.random.rand(6,6)
print('\nQR factorization of A using Modified Gram-Schmidt procedure:')
(U,R) = Modified_Gram_Schmidt(A)
print('||U*R-A|| =', np.linalg.norm(np.dot(U,R)-A,2))
print('||U^T*U-I|| =',np.linalg.norm(np.dot(U.T,U)-np.eye(6),2))
print('||U*U^T-I|| =',np.linalg.norm(np.dot(U,U.T)-np.eye(6),2))

print('\nQR factorization of A using Standard Gram-Schmidt procedure:')
(U,R) = Gram_Schmidt(A)
print('||U*R-A|| =', np.linalg.norm(np.dot(U,R)-A,2))
print('||U^T*U-I|| =',np.linalg.norm(np.dot(U.T,U)-np.eye(6),2))
print('||U*U^T-I|| =',np.linalg.norm(np.dot(U,U.T)-np.eye(6),2))


QR factorization of A using Modified Gram-Schmidt procedure:
||U*R-A|| = 1.914133529053818e-16
||U^T*U-I|| = 1.1587736454813618e-15
||U*U^T-I|| = 1.2163276376319542e-15

QR factorization of A using Standard Gram-Schmidt procedure:
||U*R-A|| = 1.5954035382302117e-16
||U^T*U-I|| = 4.579167374730279e-15
||U*U^T-I|| = 4.6818936560712015e-15
